In [1]:
### Benjamin Tollison ###
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import sympy as sp
from IPython.display import Latex, Math, display
from sympy import (
    Eq,
    Function,
    Matrix,
    cos,
    cosh,
    exp,
    integrate,
    lambdify,
    pi,
    sin,
    sinh,
    symbols,
)
from decimal import Decimal
from sympy.solvers.pde import pdsolve
from sympy.solvers.solveset import linsolve, nonlinsolve
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)
def displayVariable(variable:str,RHS):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
def displayVariableWithUnits(variable:str,RHS,units):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    latexUnit = sp.latex(symbols(units))
    display(Math(left + '=' + right + '\\;' +'\\left['+ latexUnit + '\\right]'))
def format_scientific(number:float):
    a = '%E' % number
    return a.split('E')[0].rstrip('0').rstrip('.') + 'E' + a.split('E')[1]
deg2rad = np.pi/180
rad2deg = 180/np.pi

In [2]:
A_1 = 0.12
A_2 = 0.03
v1 = 130
beta = 15*deg2rad
v2 = ((A_1/A_2)*v1**2*np.tan(beta))**0.5
v3 = (v1**2*A_1)/((v1*A_1+v2*A_2)*np.cos(beta))
displayVariableWithUnits('v_2',v2,'\\frac{m}{s}')
displayVariableWithUnits('v_3',v3,'\\frac{m}{s}')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [3]:
# question 2
v1,v2 = 70,220
sigma1,sigma2 = 0.15,0.25
T1,T2,T3 = 900,300,550
P1,P2 = 2.2*10**6,0.12*10**6
rho1 = P1/(287*T1)
rho2 = P2/(287*T2)
specific_heat_P = 1004.5
h1,h2,h3 = specific_heat_P*T1, specific_heat_P*T2, specific_heat_P*T3
mass_flow_rate_1 = rho1*v1*sigma1
mass_flow_rate_2 = rho2*v2*sigma2
mass_flow_rate_3 = mass_flow_rate_1 + mass_flow_rate_2
power = mass_flow_rate_1*h1 + mass_flow_rate_2*h2 - mass_flow_rate_3*h3
displayVariableWithUnits('\\mathcal{P}',power*10**-6,'{MW}')
T3_with_no_power = (mass_flow_rate_1*h1+mass_flow_rate_2*h2)/(specific_heat_P*mass_flow_rate_3)
displayVariableWithUnits('T_{\\mathcal{P}=0}',T3_with_no_power,'K')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [4]:
# question 3
T1 = 1400
delta_s = 45
R = 287
gamma = 1.36
T2 = T1*np.exp((delta_s+R*np.log(1.01325/18))*((gamma-1)/(gamma*R)))
displayVariableWithUnits('T_2',T2,'K')
T2 = T1*np.exp((np.log(1.01325/18))*((gamma-1)/(gamma)))
displayVariableWithUnits('T_2',T2,'K')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Question 4
I am reusing the householder method that I used to solve the general problem in 303 and below I describe how it works:\
$$\frac{\sigma}{\sigma_{cr}} = 
\frac{1}{M} \left[\frac{2}{\gamma+1}\left(1+\frac{\gamma-1}{2}M^2\right)\right]^\frac{\gamma+1}{2(\gamma-1)}$$
We can us Householder's second order root method from the nasa report:\
https://www.grc.nasa.gov/www/winddocs/utilities/b4wind_guide/mach.html
$$ x_{i+1} = x_i - \frac{2f}{f'-\sqrt{f'^2-ff''}}$$
With the subsonic regime being defined by:
$$f = (P+QX)^\frac{1}{Q} - RX = 0$$
$$f' = (P+QX)^{\frac{1}{Q}-1} - R$$
$$f'' = P(P+QX)^{\frac{1}{Q}-2}$$
and the supersonic regime can be found with the following:
$$f = (PX+Q)^\frac{1}{P} - RX = 0$$
$$f' = (PX+Q)^{\frac{1}{P}-1} - R$$
$$f'' = Q(PX+Q)^{\frac{1}{P}-2}$$
And the Coefficients can be defined by:
$$P=\frac{2}{\gamma+1}$$
$$Q = \frac{\gamma-1}{\gamma+1}=1-P $$
$$X_{subsonic} = M^2$$
$$X_{supersonic} = \frac{1}{M^2}$$
$$R_{subsonic} = \left(\frac{\sigma}{\sigma_{cr}}\right)^2$$
$$R_{supersonic} = \left(\frac{\sigma}{\sigma_{cr}}\right)^\frac{2Q}{P}$$
Therefore we can have 4 possible solutions. Subsonic->subsonic, subsonic->supersonic, supersonic->subsonic, or supersonic->supersonic

In [5]:
def HouseholdP2(x_intial:float,scheme_function,scheme_prime,scheme_double_prime)->float:
  max_iterations = 1000
  while abs(scheme_function(x_intial)) > 1e-8:
    x_intial = x_intial - ((2*scheme_function(x_intial))/(scheme_prime(x_intial) - (scheme_prime(x_intial)**2-scheme_function(x_intial)*scheme_double_prime(x_intial))**0.5))
    max_iterations -=1
    if max_iterations ==0:
      print('The scheme didn\'t converge')
      break
  return x_intial
def Householder(section_supersonic:bool,area_ratio)->float:
  P = 2/2.4
  Q = 1-P
  if section_supersonic==False:
    R = (area_ratio)**2
    a = P**(1/Q)
    r = (R-1)/(2*a)
    x_intial = 1 / ((1+r)+np.sqrt(r*(r+2)))
    f = lambda X : (P+Q*X)**(1/Q) - R*X
    f_prime = lambda X: (P+Q*X)**((1/Q)-1) - R
    f_double_prime = lambda X: P*(P+Q*X)**((1/Q)-2)
    x_final = HouseholdP2(x_intial,f,f_prime,f_double_prime)
    return (x_final)**0.5
  if section_supersonic == True:
    R = (area_ratio)**(2*Q/P)
    a = Q**(1/P)
    r = (R-1)/(2*a)
    x_intial = 1 / ((1+r)+np.sqrt(r*(r+2)))
    f = lambda X : (P*X+Q)**(1/P) - R*X
    f_prime = lambda X: (P*X+Q)**((1/P)-1) - R
    f_double_prime = lambda X: Q*(P*X+Q)**((1/P)-2)
    x_final = abs(HouseholdP2(x_intial,f,f_prime,f_double_prime))
    return 1/(x_final)**0.5
pressure_over_stagnation_function = lambda Mach_number: (1+.2*Mach_number**2)**(1.4/0.4)
area_exit = 1.53
M_subsonic = Householder(False,area_exit)
M_supersonic = Householder(True,area_exit)
displayVariable('M_{subsonic}',M_subsonic)
displayVariable('M_{supersonic}',M_supersonic)
P_subsonic = 1/pressure_over_stagnation_function(M_subsonic)
P_supersonic = 1/pressure_over_stagnation_function(M_supersonic)
displayVariableWithUnits('P_{subsonic}',P_subsonic,'bar')
displayVariableWithUnits('P_{supersonic}',P_supersonic,'bar')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
# method 2
Pressure_critical = (1 +.2)**(-1.4/.4)
displayVariable('P^*',Pressure_critical)
Pressure_exit_1 = 0.886
Pressure_exit_2 = 0.154
P_subsonic = 1/pressure_over_stagnation_function(0.42)
P_supersonic = 1/pressure_over_stagnation_function(1.88)
displayVariableWithUnits('P_{subsonic}',P_subsonic,'bar')
displayVariableWithUnits('P_{supersonic}',P_supersonic,'bar')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>